In [29]:
!rm -rf SAIRA && git clone https://github.com/kilimanj4r0/SAIRA.git

Cloning into 'SAIRA'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (1043/1043), done.
remote: Compressing objects: 100% (767/767), done.
remote: Total 1043 (delta 306), reused 994 (delta 257), pack-reused 0
Receiving objects: 100% (1043/1043), 2.06 MiB | 5.72 MiB/s, done.
Resolving deltas: 100% (306/306), done.


In [ ]:
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q bert_score

In [8]:
!pip uninstall -y preprocessing
!pip install -q -U nltk

Found existing installation: preprocessing 0.1.13
Uninstalling preprocessing-0.1.13:
  Successfully uninstalled preprocessing-0.1.13


In [36]:
import os
import json
from collections import defaultdict

import nltk

nltk.download('wordnet')
!cd /usr/share/nltk_data/corpora/ && unzip -o wordnet.zip

from evaluate import load

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  wordnet.zip
  inflating: wordnet/lexnames        
  inflating: wordnet/data.verb       
  inflating: wordnet/index.adv       
  inflating: wordnet/adv.exc         
  inflating: wordnet/index.verb      
  inflating: wordnet/cntlist.rev     
  inflating: wordnet/data.adj        
  inflating: wordnet/index.adj       
  inflating: wordnet/LICENSE         
  inflating: wordnet/citation.bib    
  inflating: wordnet/noun.exc        
  inflating: wordnet/verb.exc        
  inflating: wordnet/README          
  inflating: wordnet/index.sense     
  inflating: wordnet/data.noun       
  inflating: wordnet/data.adv        
  inflating: wordnet/index.noun      
  inflating: wordnet/adj.exc         


In [30]:
!tree /kaggle/working/SAIRA/data/results

/kaggle/working/SAIRA/data/results
├── llama2-13b-emb-base-default
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── llama2-13b-emb-base-summarize
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── mistral-instruct-7b-emb-base-default
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── mistral-instruct-7b-emb-base-summarize
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
├── orca2-13b-emb-base-default
│   ├── faq_academic.json
│   ├── faq_admin.json
│   ├── faq_extracurricular.json
│   └── faq_household.json
└── orca2-13b-emb-base-summarize
    ├── faq_academic.json
    ├── faq_admin.json
    ├── faq_extracurricular.json
    └── faq_household.json

6 directories, 24 files


In [19]:
bleu_metric = load('bleu')
rouge_metric = load('rouge')
meteor_metric = load('meteor')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [37]:
base_directory = '/kaggle/working/SAIRA/data/results'
models = os.listdir(base_directory)

# Dictionary to store scores for each model
model_scores = defaultdict(list)

for model in models:
    model_directory = os.path.join(base_directory, model)
    if os.path.isdir(model_directory):
        outputs = os.listdir(model_directory)
        
        # Temporary lists to store scores for each output of the model
        bleu_scores = []
        rouge_scores = []
        meteor_scores = []
        
        for output in outputs:
            output_file = os.path.join(model_directory, output)
            if os.path.isfile(output_file) and output.endswith('.json'):
                with open(output_file, 'r') as file:
                    data = json.load(file)
                    
                    questions = [item['question'] for item in data]
                    references = [item['answer'] for item in data]
                    predictions = [item['generated_answer'] for item in data]
                    
                    # Compute scores for the current output
                    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
                    bleu_scores.append(bleu_score['bleu'])
                    
                    rouge_score = rouge_metric.compute(predictions=predictions, references=references)
                    rouge_scores.append(rouge_score['rougeL'])
                    
                    meteor_score = meteor_metric.compute(predictions=predictions, references=references)
                    meteor_scores.append(meteor_score['meteor'])
        
        # Calculate average scores for the model
        model_scores[model] = {
            'BLEU': sum(bleu_scores) / len(bleu_scores),
            'ROUGE': sum(rouge_scores) / len(rouge_scores),
            'METEOR': sum(meteor_scores) / len(meteor_scores)
        }

# Output average scores for each model
for model, scores in model_scores.items():
    print(f"Model: {model}")
    print(f"Average BLEU score:   {scores['BLEU']:.4f}")
    print(f"Average ROUGE score:  {scores['ROUGE']:.4f}")
    print(f"Average METEOR score: {scores['METEOR']:.4f}")
    print()

Model: llama2-13b-emb-base-default
Average BLEU score:   0.0412
Average ROUGE score:  0.1534
Average METEOR score: 0.2945

Model: llama2-13b-emb-base-summarize
Average BLEU score:   0.0155
Average ROUGE score:  0.1044
Average METEOR score: 0.2205

Model: mistral-instruct-7b-emb-base-summarize
Average BLEU score:   0.0225
Average ROUGE score:  0.1276
Average METEOR score: 0.2301

Model: orca2-13b-emb-base-default
Average BLEU score:   0.0190
Average ROUGE score:  0.0946
Average METEOR score: 0.2287

Model: orca2-13b-emb-base-summarize
Average BLEU score:   0.0172
Average ROUGE score:  0.0988
Average METEOR score: 0.2258

Model: mistral-instruct-7b-emb-base-default
Average BLEU score:   0.0707
Average ROUGE score:  0.2142
Average METEOR score: 0.3020

